# PyTorch Tutorial 23: Production Inference Patterns & MLOps

**The Reality**: 87% of ML models never make it to production. The ones that do often fail within months.

**Why?** Training a model is 10% of the work. The other 90% is:
- Choosing the right inference pattern
- Monitoring for drift and degradation
- Cost optimization (inference = 90% of ML costs!)
- Safe deployments and rollbacks
- A/B testing and experimentation

This notebook teaches you **production ML engineering** - the skills that separate researchers from production ML engineers at FAANG companies.

## Learning Objectives
1. Master the **Feature/Training/Inference (FTI) Pipeline** architecture
2. Choose between **batch vs online inference** correctly
3. Implement **model monitoring** and drift detection
4. Understand **cost optimization** strategies
5. Deploy safely with **canary, blue-green**, and **A/B testing**
6. Build **production-grade MLOps** workflows

---

## Part 1: The FTI Pipeline Architecture (2025 Standard)

### Old Mental Model (Outdated)
```
Training Pipeline → Model → Serving
```
**Problem**: Treats ML as a monolith, hard to maintain

### New Mental Model: FTI Pipelines
```
┌─────────────────┐
│ Feature Pipeline│ ← Runs continuously
│  (hourly/daily) │
└────────┬────────┘
         ↓ (writes to Feature Store)
┌─────────────────┐
│Training Pipeline│ ← Runs on schedule (daily/weekly)
│ (reads features)│
└────────┬────────┘
         ↓ (publishes to Model Registry)
┌─────────────────┐
│Inference Pipeline│ ← Serves predictions
│ (reads features+model)│
└─────────────────┘
```

### Key Insight
**Three independent pipelines** sharing a common storage layer:
- **Feature Store** (features)
- **Model Registry** (models)
- **Prediction Store** (outputs)

**Benefits**:
- Features computed once, used everywhere (no training-serving skew!)
- Model updates independent of feature changes
- Easy to debug each pipeline separately
- Enables both batch AND real-time ML systems

### Training-Serving Skew (The Silent Killer)

This is the most common production ML bug and one of the hardest to detect:

**What it is**: When the features used during training differ from the features used during inference. The model was trained on one "version" of the data but sees a different version in production.

**Common causes**:
1. **Different code paths**: Training preprocesses data in Python/Pandas, serving preprocesses in Java/Go — subtle differences accumulate
2. **Time leakage**: Training uses features computed with future information that's unavailable at prediction time
3. **Stale features**: Feature store hasn't been updated, so serving uses outdated values
4. **Different library versions**: NumPy or Pandas version difference causes rounding differences

**How to prevent it**: The Feature Store pattern solves this by computing features exactly once and storing them. Both training and serving read from the same store, guaranteeing consistency.

### Feature Store Deep Dive

A Feature Store is a centralized repository that manages the lifecycle of ML features:

```
Raw Data → Feature Pipeline → Feature Store → {Training, Serving}
                                    │
                              ┌─────┴─────┐
                              │ Offline    │ ← Historical features for training
                              │ (batch)    │    (e.g., Hive/S3/BigQuery)
                              ├────────────┤
                              │ Online     │ ← Low-latency features for serving
                              │ (real-time)│    (e.g., Redis/DynamoDB)
                              └────────────┘
```

Popular Feature Stores: **Feast** (open-source), **Tecton** (managed), **Hopsworks** (open-source), **Databricks Feature Store**

## Part 2: Batch vs Online Inference - The Decision Framework

### Decision Tree
```
Do you need predictions in < 1 second?
  ├─ Yes → Online Inference
  └─ No → Can you predict all possible inputs ahead of time?
            ├─ Yes → Batch Inference (cheaper!)
            └─ No → Online Inference (long tail)
```

### Detailed Comparison

| Aspect | Batch Inference | Online Inference |
|--------|----------------|------------------|
| **Latency** | Hours to days | < 100ms |
| **Cost** | Low (scheduled) | High (always-on) |
| **Complexity** | Simple | Complex (load balancing, caching) |
| **Freshness** | Stale (hours old) | Real-time |
| **Throughput** | Very high (parallel processing) | Limited by hardware |
| **Compute** | Batch jobs (Spark, Airflow) | API servers (FastAPI, vLLM) |
| **Storage** | Database (PostgreSQL, Redis) | In-memory cache |
| **Use Cases** | Recommendations, email campaigns | Fraud detection, chatbots |

### Hybrid Pattern (Best of Both)
Many systems use BOTH:
1. **Batch**: Pre-compute predictions for common cases
2. **Online**: Fallback for cache misses or real-time needs

**Example**: YouTube recommendations
- Batch: Pre-compute top 100 videos for each user (nightly)
- Online: Rerank based on current session + real-time signals

In [ ]:
import torch
import torch.nn as nn
import time
from typing import List, Dict, Optional
import random

# Example: Hybrid Inference System
class HybridInferenceSystem:
    """
    Combines batch pre-computation with online fallback.
    Used at Netflix, YouTube, Amazon, etc.
    """
    
    def __init__(self, model, cache_size=1000):
        self.model = model
        self.cache = {}  # In production: Redis
        self.cache_size = cache_size
        self.stats = {'cache_hits': 0, 'cache_misses': 0}
    
    def batch_precompute(self, user_ids: List[str], features_batch: torch.Tensor):
        """Batch job: Pre-compute predictions for known users."""
        with torch.no_grad():
            predictions = self.model(features_batch)
        
        # Store in cache
        for user_id, pred in zip(user_ids, predictions):
            self.cache[user_id] = pred.tolist()
        
        print(f"✅ Pre-computed {len(user_ids)} predictions")
    
    def online_predict(self, user_id: str, features: Optional[torch.Tensor] = None) -> List[float]:
        """Online API: Get prediction (cache or compute)."""
        # Try cache first
        if user_id in self.cache:
            self.stats['cache_hits'] += 1
            return self.cache[user_id]  # < 1ms latency!
        
        # Cache miss: Compute on-the-fly
        self.stats['cache_misses'] += 1
        if features is None:
            features = torch.randn(1, 10)  # In production: fetch from feature store
        
        with torch.no_grad():
            prediction = self.model(features)
        
        result = prediction[0].tolist()
        
        # Update cache
        if len(self.cache) < self.cache_size:
            self.cache[user_id] = result
        
        return result
    
    def get_cache_hit_rate(self) -> float:
        total = self.stats['cache_hits'] + self.stats['cache_misses']
        return self.stats['cache_hits'] / total if total > 0 else 0.0

# Demo
class DemoModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(10, 5)
    
    def forward(self, x):
        return self.fc(x)

model = DemoModel()
model.eval()
system = HybridInferenceSystem(model)

# Step 1: Batch pre-computation (runs nightly)
print("\n📦 BATCH JOB: Pre-computing predictions...\n")
common_users = [f"user_{i}" for i in range(100)]
batch_features = torch.randn(100, 10)
system.batch_precompute(common_users, batch_features)

# Step 2: Online requests
print("\n⚡ ONLINE API: Serving requests...\n")
for i in range(10):
    # Mix of common (cached) and rare (uncached) users
    if random.random() < 0.8:  # 80% are cached
        user_id = f"user_{random.randint(0, 99)}"
    else:  # 20% are new/rare users
        user_id = f"user_new_{i}"
    
    start = time.time()
    pred = system.online_predict(user_id)
    latency = (time.time() - start) * 1000
    
    cached = "✅ CACHE" if user_id in system.cache else "❌ COMPUTE"
    print(f"{cached} | {user_id} | {latency:.2f}ms")

print(f"\n📊 Cache hit rate: {system.get_cache_hit_rate()*100:.1f}%")
print("💡 In production, 90%+ hit rate saves millions in compute costs!")

## Part 3: Cost Optimization Strategies

### The Economics of Inference
**Fact**: Inference costs account for **90% of production ML expenses**.

For a model serving 1M requests/day:
- **Naive deployment**: $50,000/month
- **Optimized deployment**: $5,000/month

**10x cost reduction** is possible with these techniques:

### 1. Model Optimization
- **Quantization**: INT8 → 4x smaller, 2-3x faster
- **Distillation**: Train small model to mimic large one (10x cheaper)
- **Pruning**: Remove unimportant weights (30-50% speedup)

### 2. Batching
- **Static batching**: Wait to fill batch → 5-10x throughput
- **Dynamic batching**: Flexible batch size → 3-5x throughput
- **Micro-batching**: Small batches (e.g., 10ms window) → balance latency/throughput

### 3. Caching
- **Result caching**: Cache predictions for common inputs
- **Feature caching**: Avoid recomputing features
- **KV cache**: For LLMs (see Notebook 21)

### 4. Hardware Selection
- **CPU vs GPU**: Use CPU for <50 QPS, GPU for >50 QPS
- **Spot instances**: 70% cost savings (with fault tolerance)
- **Inference-optimized chips**: AWS Inferentia, Google TPU Edge

### 5. Autoscaling
- Scale down during low traffic (nights, weekends)
- **Kubernetes HPA**: Scale based on QPS or latency
- **Serverless**: AWS Lambda for bursty workloads

### Cost Breakdown Example (Llama-3 8B)
```
Option 1: Naive (HuggingFace + A100)
- 50 QPS @ $3/hour = $2,160/month

Option 2: vLLM + Batching (A100)
- 150 QPS @ $3/hour = $720/month (3x cheaper!)

Option 3: vLLM + Quantization + Spot (A10g)
- 100 QPS @ $0.50/hour = $360/month (6x cheaper!)
```

## Part 4: Model Monitoring & Drift Detection

### What to Monitor

#### 1. Input Distribution Drift (Data Drift)
Are the inputs changing over time? For example, if your fraud detection model was trained on transactions averaging $50, but now the average is $500 (holiday season), the model may perform poorly.

**Types of drift**:
- **Covariate shift**: Input distribution changes (most common)
- **Concept drift**: The relationship between inputs and outputs changes (e.g., "fraud patterns" evolve)
- **Label drift**: The distribution of labels changes (e.g., fraud rate increases)

#### 2. Output Distribution Drift  
Are the predictions changing? A sudden shift in prediction distribution (e.g., the model predicts "fraud" 10x more than yesterday) indicates something is wrong — either the inputs changed or the model is malfunctioning.

#### 3. Performance Metrics
When labels arrive, how accurate is the model? In many production systems, ground truth labels arrive with a delay (e.g., you know if a transaction was actually fraud only after investigation, which could take days).

#### 4. System Metrics
Latency (P50, P95, P99), throughput (requests/sec), error rate, GPU utilization, memory usage.

### PSI (Population Stability Index) — The Standard Drift Metric

PSI compares two distributions by measuring how much one has "shifted" from the other:

$$ PSI = \sum_{i=1}^{N} (p_i^{current} - p_i^{baseline}) \times \ln\left(\frac{p_i^{current}}{p_i^{baseline}}\right) $$

Interpretation thresholds:
- **PSI < 0.1**: No drift — distributions are stable
- **PSI 0.1 - 0.2**: Moderate drift — investigate, consider retraining
- **PSI > 0.2**: Significant drift — retrain the model

**Why PSI over simpler metrics?** A simple mean/variance comparison can miss distribution shape changes. PSI compares the full histogram, catching subtle shifts like bimodal distributions or tail changes.

In [ ]:
import numpy as np
from collections import deque

class DriftDetector:
    """Detect distribution drift using Population Stability Index (PSI)."""
    
    def __init__(self, baseline_data, bins=10, threshold=0.2):
        self.bins = bins
        self.threshold = threshold
        
        # Compute baseline distribution
        self.baseline_hist, self.bin_edges = np.histogram(
            baseline_data, bins=bins
        )
        self.baseline_hist = self.baseline_hist / len(baseline_data)  # Normalize
    
    def compute_psi(self, current_data):
        """
        Compute Population Stability Index.
        PSI < 0.1: No drift
        PSI 0.1-0.2: Moderate drift
        PSI > 0.2: Significant drift (retrain!)
        """
        current_hist, _ = np.histogram(current_data, bins=self.bin_edges)
        current_hist = current_hist / len(current_data)
        
        # Avoid log(0)
        current_hist = np.clip(current_hist, 1e-10, 1)
        baseline_hist = np.clip(self.baseline_hist, 1e-10, 1)
        
        psi = np.sum(
            (current_hist - baseline_hist) * np.log(current_hist / baseline_hist)
        )
        
        return psi
    
    def check_drift(self, current_data):
        psi = self.compute_psi(current_data)
        has_drift = psi > self.threshold
        
        if has_drift:
            return True, f"🚨 DRIFT DETECTED! PSI = {psi:.3f}"
        else:
            return False, f"✅ No drift. PSI = {psi:.3f}"

# Demo: Simulating drift over time
print("\n📊 Drift Detection Demo\n")

# Baseline: Training data distribution
baseline = np.random.normal(0, 1, 10000)
detector = DriftDetector(baseline, threshold=0.2)

# Simulate production data over time
scenarios = [
    ("Week 1", np.random.normal(0, 1, 1000)),      # No drift
    ("Week 2", np.random.normal(0.1, 1, 1000)),    # Slight drift
    ("Week 3", np.random.normal(0.5, 1.2, 1000)),  # Moderate drift
    ("Week 4", np.random.normal(2, 2, 1000)),      # Severe drift!
]

for period, data in scenarios:
    has_drift, message = detector.check_drift(data)
    print(f"{period}: {message}")
    if has_drift:
        print("   → Recommend: Retrain model with recent data")

print("\n💡 In production, automate this check daily and trigger retraining!")

## Part 5: Safe Deployment Patterns

### Pattern 1: Canary Deployment
Gradually roll out new model to small percentage of traffic.

```
5% traffic → New Model v2 → Monitor for 1 hour
            ↓ (if good)
20% traffic → New Model v2 → Monitor for 1 hour
            ↓ (if good)
100% traffic → New Model v2
```

**When to use**: Default choice for most production ML deployments. The gradual rollout catches problems early before they affect all users.

**What to monitor during canary**: Compare error rates, latency, and business metrics (revenue, engagement) between canary and control traffic. If the canary shows degradation, roll back immediately.

### Pattern 2: Blue-Green Deployment
Run two environments, switch traffic instantly.

```
Blue (Old Model v1) ← 100% traffic
Green (New Model v2) ← 0% traffic (testing)
            ↓ (when ready)
Blue ← 0% traffic
Green ← 100% traffic (instant switch!)
```

**Benefit**: Instant rollback if issues detected — just switch traffic back to Blue.

**When to use**: When you need zero-downtime deployment and fast rollback. More expensive (2x infrastructure) but safer for critical systems.

### Pattern 3: Shadow Deployment
Run the new model alongside production without serving its predictions to users.

```
100% traffic → Model v1 → Serve to users
      └─────→ Model v2 → Log predictions (but don't serve)
```

Compare predictions offline. Only promote v2 when its logged predictions consistently outperform v1.

**When to use**: High-risk models (medical, financial) where you need extensive validation before serving.

### Pattern 4: A/B Testing
Run two models simultaneously, compare business metrics.

```
50% users → Model A → Track revenue, engagement
50% users → Model B → Track revenue, engagement
            ↓ (after 1-2 weeks)
Choose winner based on business metrics
```

**Key**: Don't just compare model metrics (AUC, F1) — compare business metrics (revenue, engagement)! A model with lower AUC might actually drive more revenue due to better calibration or different failure modes.

### Statistical Rigor in A/B Tests

Common mistakes that lead to wrong conclusions:
1. **Peeking**: Checking results daily and stopping when p < 0.05. This inflates false positive rate from 5% to 25%+. Use sequential testing or wait for the predetermined sample size.
2. **Underpowered tests**: Not enough samples to detect small effects. Calculate required sample size before starting.
3. **Multiple comparisons**: Testing 20 metrics and declaring victory when one is significant. Apply Bonferroni correction.
4. **Network effects**: User A's experience changes user B's behavior (social networks). Use cluster randomization.

In [ ]:
# Example: Simple A/B Testing Framework
class ABTestingFramework:
    def __init__(self, model_a, model_b, split_ratio=0.5):
        self.model_a = model_a  # Control
        self.model_b = model_b  # Treatment
        self.split_ratio = split_ratio
        
        self.metrics_a = {'predictions': [], 'latencies': []}
        self.metrics_b = {'predictions': [], 'latencies': []}
    
    def route_request(self, user_id: str, features: torch.Tensor):
        """Route user to model A or B based on user_id hash."""
        # Deterministic assignment based on user_id
        user_hash = hash(user_id) % 100
        use_model_b = (user_hash / 100) < self.split_ratio
        
        start = time.time()
        
        if use_model_b:
            with torch.no_grad():
                pred = self.model_b(features)
            latency = time.time() - start
            self.metrics_b['predictions'].append(pred.item())
            self.metrics_b['latencies'].append(latency)
            variant = 'B'
        else:
            with torch.no_grad():
                pred = self.model_a(features)
            latency = time.time() - start
            self.metrics_a['predictions'].append(pred.item())
            self.metrics_a['latencies'].append(latency)
            variant = 'A'
        
        return {
            'prediction': pred.item(),
            'variant': variant,
            'latency_ms': latency * 1000
        }
    
    def get_summary(self):
        return {
            'Model A': {
                'requests': len(self.metrics_a['predictions']),
                'avg_prediction': np.mean(self.metrics_a['predictions']),
                'avg_latency_ms': np.mean(self.metrics_a['latencies']) * 1000
            },
            'Model B': {
                'requests': len(self.metrics_b['predictions']),
                'avg_prediction': np.mean(self.metrics_b['predictions']),
                'avg_latency_ms': np.mean(self.metrics_b['latencies']) * 1000
            }
        }

# Demo
model_a = DemoModel()  # Control
model_b = DemoModel()  # Treatment (slightly modified)
model_a.eval()
model_b.eval()

ab_test = ABTestingFramework(model_a, model_b, split_ratio=0.5)

print("\n🔬 A/B Test Simulation\n")
for i in range(100):
    user_id = f"user_{random.randint(1, 1000)}"
    features = torch.randn(1, 10)
    result = ab_test.route_request(user_id, features)

summary = ab_test.get_summary()
print("\n📊 A/B Test Results:\n")
for model_name, metrics in summary.items():
    print(f"{model_name}:")
    print(f"  Requests: {metrics['requests']}")
    print(f"  Avg Prediction: {metrics['avg_prediction']:.4f}")
    print(f"  Avg Latency: {metrics['avg_latency_ms']:.2f}ms")
    print()

print("💡 In production, track business metrics (revenue, CTR, engagement) not just model metrics!")

## Part 6: Production MLOps Checklist

### Pre-Deployment
- [ ] Model performance validated on holdout set
- [ ] Latency tested under load (stress testing)
- [ ] Feature consistency verified (training vs serving)
- [ ] Model size optimized (quantization, pruning)
- [ ] Rollback plan documented
- [ ] Monitoring dashboards created
- [ ] Alerts configured (latency, error rate, drift)

### Deployment
- [ ] Use canary or blue-green deployment
- [ ] Start with 5% traffic
- [ ] Monitor for 1-2 hours before scaling up
- [ ] Have kill switch ready (instant rollback)
- [ ] Document deployment in runbook

### Post-Deployment
- [ ] Monitor drift daily (PSI, KL divergence)
- [ ] Track business metrics (revenue, engagement)
- [ ] Collect feedback labels for model retraining
- [ ] Schedule retraining (weekly/monthly)
- [ ] Review costs and optimize
- [ ] Conduct post-mortem if issues occurred

### Continuous Improvement
- [ ] A/B test new model versions
- [ ] Experiment with different architectures
- [ ] Optimize inference costs (batching, caching)
- [ ] Update feature engineering
- [ ] Retrain on fresh data
- [ ] Archive old model versions

## Summary: Production ML Engineering in 2025

### Core Principles
1. **FTI Pipeline Architecture**: Separate feature, training, and inference pipelines
2. **Choose the Right Pattern**: Batch for cost, online for latency
3. **Monitor Everything**: Drift, performance, costs
4. **Deploy Safely**: Canary, blue-green, A/B testing
5. **Optimize Costs**: Inference is 90% of expenses!

### Technology Stack
- **Feature Store**: Feast, Tecton
- **Model Registry**: MLflow, Weights & Biases
- **Orchestration**: Airflow, Prefect, Dagster
- **Monitoring**: Prometheus, Grafana, Evidently
- **Deployment**: Kubernetes, Docker
- **Serving**: FastAPI, vLLM, TensorRT-LLM

### What FAANG Expects
✅ Understanding of FTI pipeline architecture
✅ Ability to choose batch vs online correctly
✅ Experience with monitoring and drift detection
✅ Knowledge of cost optimization strategies
✅ Familiarity with safe deployment patterns
✅ A/B testing and experimentation skills
✅ Production debugging and incident response

### Anti-Patterns to Avoid
❌ Deploying without monitoring
❌ Ignoring training-serving skew
❌ 100% rollout without testing
❌ Not tracking business metrics
❌ Neglecting cost optimization
❌ No rollback plan

### Further Reading
- [MLOps Principles (Google)](https://cloud.google.com/architecture/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning)
- [Chip Huyen: Designing Machine Learning Systems](https://www.oreilly.com/library/view/designing-machine-learning/9781098107956/)
- [FTI Pipeline Architecture (Hopsworks)](https://www.hopsworks.ai/post/mlops-to-ml-systems-with-fti-pipelines)

**You now have production ML engineering skills! 🚀**